In [1]:
!pip install syft

     |████████████████████████████████| 378kB 5.2MB/s 
     |████████████████████████████████| 450kB 17.1MB/s 
     |████████████████████████████████| 2.1MB 25.7MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 204kB 41.5MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 2.0MB 48.2MB/s 
     |████████████████████████████████| 753.4MB 20kB/s 
     |████████████████████████████████| 4.0MB 46.2MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 1.8MB 44.6MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 81kB 10.5MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 36.9MB 118kB/s 
     |████████████████████████████████| 2.7MB 42.7MB/s 
     |████████████████████████████████| 1.3MB 35.3MB/s 
     |

In [1]:
!pip install --upgrade PIL

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [4]:
!git clone https://github.com/tusharsoni08/Openmined-FL-Recommendation-System.git

Cloning into 'Openmined-FL-Recommendation-System'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 21 (delta 6), reused 15 (delta 3), pack-reused 0
Unpacking objects: 100% (21/21), 630.86 KiB | 716.00 KiB/s, done.


In [8]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import syft as sy
import time
from torch.utils.data import Dataset

hook = sy.TorchHook(torch)
class Arguments():
    def __init__(self):
        self.batch_size = 1
        self.test_batch_size = 50
        self.seed = 1
#bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
#alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

In [9]:
import torch
import random
import pandas as pd
from copy import deepcopy
from torch.utils.data import DataLoader, Dataset
 
random.seed(0)
 
 
class UserItemRatingDataset(Dataset):
    """Wrapper, convert <user, item, rating> Tensor into Pytorch Dataset"""
    def __init__(self, user_tensor, item_tensor, target_tensor):
        """
        args:
            target_tensor: torch.Tensor, the corresponding rating for <user, item> pair
        """
        self.user_tensor = user_tensor
        self.item_tensor = item_tensor
        self.target_tensor = target_tensor
 
    def __getitem__(self, index):
        data_tensor=torch.stack([self.user_tensor[index], self.item_tensor[index]],dim=0)
        # print(data_tensor.shape)
        return data_tensor, self.target_tensor[index]
 
    def __len__(self):
        return len(self.user_tensor)

In [10]:
# sample_generator = SampleGenerator(ratings=ml1m_rating)
# # base_federated=dataset.federate((bob, alice))
# base = sample_generator.instance_a_train_loader(4, 32)
rs_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
train_data = pd.read_csv('Openmined-FL-Recommendation-System/Data/ua.base', sep='\t', names=rs_cols, encoding='latin-1')
user_ids = train_data["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
movie_ids = train_data["movie_id"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
train_data["user"] = train_data["user_id"].map(user2user_encoded)
train_data["movie"] = train_data["movie_id"].map(movie2movie_encoded)

num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)
train_data["rating"] = train_data["rating"].values.astype(np.float32)
# federated_SVHN=UserItemRatingDataset(torch.LongTensor(train_data["user"]),torch.LongTensor(train_data["movie"]),torch.FloatTensor(train_data["rating"]))


In [10]:
#workers=[bob,alice]
#federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
#                         UserItemRatingDataset(torch.LongTensor(train_data["user"]),torch.LongTensor(train_data["movie"]),torch.FloatTensor(train_data["rating"])).federate(tuple(workers)),batch_size=1024,shuffle=True,iter_per_worker=True)

In [12]:
#Create virtual workers

workers = []

for user_id in user_ids:
    worker = sy.VirtualWorker(hook, id="user_"+str(user_id))
    workers.append(worker)

federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
                         UserItemRatingDataset(torch.LongTensor(train_data["user"]),torch.LongTensor(train_data["movie"]),torch.FloatTensor(train_data["rating"])).federate(tuple(workers)),batch_size=1024,shuffle=True,iter_per_worker=True)

In [13]:
class Model(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size):
        super(Model, self).__init__()
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)
        
        self.fc_layers = nn.ModuleList()
        self.fc_layers.append(nn.Linear(50, 50))
        self.output_layer = nn.Linear(embedding_size, 1)

    def forward(self, train_data):
        users = torch.as_tensor(train_data[:,0])
        movies = torch.as_tensor(train_data[:,1])
        user_embedding_x = self.user_embedding(users)
        movie_embedding_y = self.movie_embedding(movies)
        prod=torch.mul(user_embedding_x,movie_embedding_y)
        # concatenate user and movie embeddings to form input to 1 dim
        # x = torch.cat([user_embedding_x, movie_embedding_y], 1)
        
        # for idx, _ in enumerate(range(len(self.fc_layers))):
        #     x = self.fc_layers[idx](x)
        #     x = F.dropout(x, p=0.2)
        #     x = F.batch_norm(x)
        #     x = F.relu(x)
        
        logit = self.output_layer(prod)
        # The sigmoid activation forces the rating to between 0 and 1
        rating = torch.sigmoid(logit)
        return rating


In [ ]:
# from syft.frameworks.torch.fl import utils
# embed_size=50
# model = Model(num_users,num_movies,embed_size)
# optimizer = optim.SGD(model.parameters(), lr=0.1)
# lr=0.1
# def trainn():
#     for epoch in range(0, 5):
#         for batch_idx, (data, target) in enumerate(federated_train_loader):
#             print(data.location)
#             print(target.location)
#             # send the model to the client device where the data is present
#             model.send(data.location)
#             # training the model
#             optimizer.zero_grad()
#             prediction = model(data)
#             loss = F.mse_loss(prediction.view(-1), target)
#             loss.backward()
#             optimizer.step()
#             # get back the improved model
#             print(loss.get())
#             model.get()
#             return utils.federated_avg({
#                 "model": model
#             })

In [ ]:

# for epoch in range(5):
#     start_time = time.time()
#     print("Epoch Number {epoch + 1}")
#     federated_model = trainn()
#     model = federated_model
# #     test(federated_model)
#     total_time = time.time() - start_time
#     print('Communication time over the network', round(total_time, 2), 's\n')

In [14]:
from syft.frameworks.torch.fl import utils
import torch.nn as nn
import torch.nn.functional as f
def train_on_batches(worker, batches, model_in, device, lr):
    """Train the model on the worker on the provided batches
    Args:
        worker(syft.workers.BaseWorker): worker on which the
        training will be executed
        batches: batches of data of this worker
        model_in: machine learning model, training will be done on a copy
        device (torch.device): where to run the training
        lr: learning rate of the training steps
    Returns:
        model, loss: obtained model and loss after training
    """
    model = model_in.copy()
    optimizer = optim.SGD(model.parameters(), lr=lr)  # TODO momentum is not supported at the moment

    model.train()
    model.send(worker)
    loss_local = False
    LOG_INTERVAL=25

    for batch_idx, (data, target) in enumerate(batches):
        loss_local = False
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = f.mse_loss(output.view(-1), target)
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            loss = loss.get()  # <-- NEW: get the loss back
            loss_local = True
            print(
                "Train Worker {}: [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    worker.id,
                    batch_idx,
                    len(batches),
                    100.0 * batch_idx / len(batches),
                    loss.item(),
                )
            )

    if not loss_local:
      loss = loss.get()  # <-- NEW: get the loss back
    model.get()  # <-- NEW: get the model back
    return model, loss


def get_next_batches(fdataloader: sy.FederatedDataLoader, nr_batches: int):
    """retrieve next nr_batches of the federated data loader and group
    the batches by worker
    Args:
        fdataloader (sy.FederatedDataLoader): federated data loader
        over which the function will iterate
        nr_batches (int): number of batches (per worker) to retrieve
    Returns:
        Dict[syft.workers.BaseWorker, List[batches]]
    """
    batches = {}
    for worker_id in fdataloader.workers:
        worker = fdataloader.federated_dataset.datasets[worker_id].location
        batches[worker] = []
    try:
        for i in range(nr_batches):
            next_batches = next(fdataloader)
            for worker in next_batches:
                batches[worker].append(next_batches[worker])
    except StopIteration:
        pass
    return batches


def train(
    model, device, federated_train_loader, lr, federate_after_n_batches, abort_after_one=False
):
    model.train()

    nr_batches = federate_after_n_batches

    models = {}
    loss_values = {}

    iter(federated_train_loader)  # initialize iterators
    batches = get_next_batches(federated_train_loader, nr_batches)
    counter = 0

    while True:
        print(f"Starting training round, batches [{counter}, {counter + nr_batches}]")
        data_for_all_workers = True
        for worker in batches:
            curr_batches = batches[worker]
            if curr_batches:
                models[worker], loss_values[worker] = train_on_batches(
                    worker, curr_batches, model, device, lr
                )
            else:
                data_for_all_workers = False
        counter += nr_batches
        if not data_for_all_workers:
            print("At least one worker ran out of data, stopping.")
            break

        model = utils.federated_avg(models)
        batches = get_next_batches(federated_train_loader, nr_batches)
        if abort_after_one:
            break
    return model


In [15]:
embed_size=50
model = Model(num_users,num_movies,embed_size)
optimizer = optim.SGD(model.parameters(), lr=0.1)
lr=0.1
federate_after_n_batches=50
device=torch.device("cpu")
for epoch in range(1, 5):
        print("Starting epoch {}/{}".format(epoch,5))
        model = train(model, device, federated_train_loader, lr, federate_after_n_batches)


Starting epoch 1/5
Starting training round, batches [0, 50]
Train Worker user_1: [0/1 (0%)]	Loss: 11.610948
Train Worker user_2: [0/1 (0%)]	Loss: 12.028971
Train Worker user_3: [0/1 (0%)]	Loss: 10.711548
Train Worker user_4: [0/1 (0%)]	Loss: 9.428104
Train Worker user_5: [0/1 (0%)]	Loss: 8.340612
Train Worker user_6: [0/1 (0%)]	Loss: 8.466829
Train Worker user_7: [0/1 (0%)]	Loss: 9.085833
Train Worker user_8: [0/1 (0%)]	Loss: 13.865302
Train Worker user_9: [0/1 (0%)]	Loss: 14.810122
Train Worker user_10: [0/1 (0%)]	Loss: 13.069444
Train Worker user_11: [0/1 (0%)]	Loss: 11.581843
Train Worker user_12: [0/1 (0%)]	Loss: 12.834407
Train Worker user_13: [0/1 (0%)]	Loss: 14.090590
Train Worker user_14: [0/1 (0%)]	Loss: 13.588277
Train Worker user_15: [0/1 (0%)]	Loss: 10.072558
Train Worker user_16: [0/1 (0%)]	Loss: 10.464640
Train Worker user_17: [0/1 (0%)]	Loss: 11.153332
Train Worker user_18: [0/1 (0%)]	Loss: 10.725760
Train Worker user_19: [0/1 (0%)]	Loss: 6.666382
Train Worker user_20: [

Train Worker user_169: [0/1 (0%)]	Loss: 12.741108
Train Worker user_170: [0/1 (0%)]	Loss: 11.512078
Train Worker user_171: [0/1 (0%)]	Loss: 14.312288
Train Worker user_172: [0/1 (0%)]	Loss: 10.143112
Train Worker user_173: [0/1 (0%)]	Loss: 8.741161
Train Worker user_174: [0/1 (0%)]	Loss: 8.576496
Train Worker user_175: [0/1 (0%)]	Loss: 7.267595
Train Worker user_176: [0/1 (0%)]	Loss: 7.483057
Train Worker user_177: [0/1 (0%)]	Loss: 7.536872
Train Worker user_178: [0/1 (0%)]	Loss: 9.383882
Train Worker user_179: [0/1 (0%)]	Loss: 8.263063
Train Worker user_180: [0/1 (0%)]	Loss: 8.835452
Train Worker user_181: [0/1 (0%)]	Loss: 9.001050
Train Worker user_182: [0/1 (0%)]	Loss: 14.721501
Train Worker user_183: [0/1 (0%)]	Loss: 11.232083
Train Worker user_184: [0/1 (0%)]	Loss: 8.313685
Train Worker user_185: [0/1 (0%)]	Loss: 7.815060
Train Worker user_186: [0/1 (0%)]	Loss: 6.944493
Train Worker user_187: [0/1 (0%)]	Loss: 7.248990
Train Worker user_188: [0/1 (0%)]	Loss: 8.724638
Train Worker u

Train Worker user_337: [0/1 (0%)]	Loss: 13.171056
Train Worker user_338: [0/1 (0%)]	Loss: 15.972045
Train Worker user_339: [0/1 (0%)]	Loss: 13.987126
Train Worker user_340: [0/1 (0%)]	Loss: 14.127873
Train Worker user_341: [0/1 (0%)]	Loss: 10.735398
Train Worker user_342: [0/1 (0%)]	Loss: 9.828887
Train Worker user_343: [0/1 (0%)]	Loss: 10.665051
Train Worker user_344: [0/1 (0%)]	Loss: 9.084096
Train Worker user_345: [0/1 (0%)]	Loss: 8.460846
Train Worker user_346: [0/1 (0%)]	Loss: 12.398295
Train Worker user_347: [0/1 (0%)]	Loss: 14.349986
Train Worker user_348: [0/1 (0%)]	Loss: 12.827155
Train Worker user_349: [0/1 (0%)]	Loss: 13.133482
Train Worker user_350: [0/1 (0%)]	Loss: 11.785542
Train Worker user_351: [0/1 (0%)]	Loss: 9.124625
Train Worker user_352: [0/1 (0%)]	Loss: 14.220974
Train Worker user_353: [0/1 (0%)]	Loss: 13.606317
Train Worker user_354: [0/1 (0%)]	Loss: 12.712528
Train Worker user_355: [0/1 (0%)]	Loss: 10.719818
Train Worker user_356: [0/1 (0%)]	Loss: 11.670859
Trai

Train Worker user_503: [0/1 (0%)]	Loss: 13.874257
Train Worker user_504: [0/1 (0%)]	Loss: 14.878921
Train Worker user_505: [0/1 (0%)]	Loss: 13.076573
Train Worker user_506: [0/1 (0%)]	Loss: 13.326321
Train Worker user_507: [0/1 (0%)]	Loss: 9.955367
Train Worker user_508: [0/1 (0%)]	Loss: 10.953760
Train Worker user_509: [0/1 (0%)]	Loss: 10.485773
Train Worker user_510: [0/1 (0%)]	Loss: 9.783897
Train Worker user_511: [0/1 (0%)]	Loss: 10.483904
Train Worker user_512: [0/1 (0%)]	Loss: 9.862290
Train Worker user_513: [0/1 (0%)]	Loss: 13.412974
Train Worker user_514: [0/1 (0%)]	Loss: 10.498102
Train Worker user_515: [0/1 (0%)]	Loss: 7.794221
Train Worker user_516: [0/1 (0%)]	Loss: 11.730251
Train Worker user_517: [0/1 (0%)]	Loss: 10.421796
Train Worker user_518: [0/1 (0%)]	Loss: 10.207066
Train Worker user_519: [0/1 (0%)]	Loss: 8.889425
Train Worker user_520: [0/1 (0%)]	Loss: 11.787768
Train Worker user_521: [0/1 (0%)]	Loss: 9.199103
Train Worker user_522: [0/1 (0%)]	Loss: 12.386455
Train 

Train Worker user_669: [0/1 (0%)]	Loss: 6.648588
Train Worker user_670: [0/1 (0%)]	Loss: 8.564942
Train Worker user_671: [0/1 (0%)]	Loss: 6.335907
Train Worker user_672: [0/1 (0%)]	Loss: 4.233161
Train Worker user_673: [0/1 (0%)]	Loss: 11.720922
Train Worker user_674: [0/1 (0%)]	Loss: 7.987923
Train Worker user_675: [0/1 (0%)]	Loss: 7.246175
Train Worker user_676: [0/1 (0%)]	Loss: 6.812571
Train Worker user_677: [0/1 (0%)]	Loss: 7.103153
Train Worker user_678: [0/1 (0%)]	Loss: 4.926912
Train Worker user_679: [0/1 (0%)]	Loss: 6.059407
Train Worker user_680: [0/1 (0%)]	Loss: 6.199393
Train Worker user_681: [0/1 (0%)]	Loss: 8.513047
Train Worker user_682: [0/1 (0%)]	Loss: 11.214540
Train Worker user_683: [0/1 (0%)]	Loss: 11.610638
Train Worker user_684: [0/1 (0%)]	Loss: 6.757172
Train Worker user_685: [0/1 (0%)]	Loss: 5.851229
Train Worker user_686: [0/1 (0%)]	Loss: 9.597483
Train Worker user_687: [0/1 (0%)]	Loss: 11.573885
Train Worker user_688: [0/1 (0%)]	Loss: 9.703388
Train Worker use

Train Worker user_837: [0/1 (0%)]	Loss: 11.000288
Train Worker user_838: [0/1 (0%)]	Loss: 7.648358
Train Worker user_839: [0/1 (0%)]	Loss: 8.444459
Train Worker user_840: [0/1 (0%)]	Loss: 16.008856
Train Worker user_841: [0/1 (0%)]	Loss: 16.472710
Train Worker user_842: [0/1 (0%)]	Loss: 12.036976
Train Worker user_843: [0/1 (0%)]	Loss: 7.612730
Train Worker user_844: [0/1 (0%)]	Loss: 9.190557
Train Worker user_845: [0/1 (0%)]	Loss: 8.698093
Train Worker user_846: [0/1 (0%)]	Loss: 7.648942
Train Worker user_847: [0/1 (0%)]	Loss: 10.053181
Train Worker user_848: [0/1 (0%)]	Loss: 12.399800
Train Worker user_849: [0/1 (0%)]	Loss: 15.874067
Train Worker user_850: [0/1 (0%)]	Loss: 10.667508
Train Worker user_851: [0/1 (0%)]	Loss: 12.420709
Train Worker user_852: [0/1 (0%)]	Loss: 13.333457
Train Worker user_853: [0/1 (0%)]	Loss: 9.575579
Train Worker user_854: [0/1 (0%)]	Loss: 7.056700
Train Worker user_855: [0/1 (0%)]	Loss: 13.272142
Train Worker user_856: [0/1 (0%)]	Loss: 8.905868
Train Wor

Train Worker user_68: [0/1 (0%)]	Loss: 11.627874
Train Worker user_69: [0/1 (0%)]	Loss: 10.791162
Train Worker user_70: [0/1 (0%)]	Loss: 9.646820
Train Worker user_71: [0/1 (0%)]	Loss: 10.851661
Train Worker user_72: [0/1 (0%)]	Loss: 12.031552
Train Worker user_73: [0/1 (0%)]	Loss: 10.437098
Train Worker user_74: [0/1 (0%)]	Loss: 9.671412
Train Worker user_75: [0/1 (0%)]	Loss: 10.447939
Train Worker user_76: [0/1 (0%)]	Loss: 11.169834
Train Worker user_77: [0/1 (0%)]	Loss: 8.100837
Train Worker user_78: [0/1 (0%)]	Loss: 9.255187
Train Worker user_79: [0/1 (0%)]	Loss: 9.031264
Train Worker user_80: [0/1 (0%)]	Loss: 10.524387
Train Worker user_81: [0/1 (0%)]	Loss: 9.910714
Train Worker user_82: [0/1 (0%)]	Loss: 10.396316
Train Worker user_83: [0/1 (0%)]	Loss: 9.960526
Train Worker user_84: [0/1 (0%)]	Loss: 12.586024
Train Worker user_85: [0/1 (0%)]	Loss: 10.522939
Train Worker user_86: [0/1 (0%)]	Loss: 13.694932
Train Worker user_87: [0/1 (0%)]	Loss: 14.599705
Train Worker user_88: [0/1 

KeyboardInterrupt: 